In [1]:
# load packages

import boto3
import botocore
import psycopg2
import sqlalchemy
import pandas as pd
import numpy as np
import time
import pickle

# Gensim uses Python’s standard logging module to log various stuff at various priority levels; to activate logging (this is optional), run
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
# Let's use Amazon S3
client = boto3.client('s3')
s3 = boto3.resource('s3')

# Print out bucket names
for bucket in s3.buckets.all():
    print(bucket.name)
bucket_name = 'cse6242oan-xchen668'

2018-11-04 16:57:47,432 : INFO : Found credentials in shared credentials file: ~/.aws/credentials
2018-11-04 16:57:47,475 : INFO : Starting new HTTPS connection (1): s3.amazonaws.com


cse6242oan-xchen668
cse6242oan-xchen668-logging
frsgr


In [20]:
restdata = pd.read_csv('review_rest.csv', quoting=3, sep=',', escapechar='\\', encoding='utf-8')

print(restdata.shape)


(1988859, 10)


In [21]:
restdata['text'] = restdata['text'].astype(str)
print(restdata.dtypes)
print(restdata.head)

review_id       object
user_id         object
business_id     object
stars          float64
date            object
text            object
useful         float64
funny          float64
cool           float64
city            object
dtype: object
<bound method NDFrame.head of                       review_id                 user_id  \
0        2gSyGZb1NAd3LJWjC-9mig  7RbqdGGd4oalu7l6WDZngw   
1        bYEweGShFgxMjGvnyFHR4g  BaQAHMOAvJQjrhb5ba59Ew   
2        c6fLaLolxXMJ6dxDMqHM7g  2JTQi5Nj3irAknH9X1sCEQ   
3        lE5gKFLCHp9cZshgiGaKsA  mLp-1_fcKsU7hx4jtPnMOA   
4        lz3qRlJU5E75a0qfH2Soew  voFM9PbMhQMm621EtPSGIg   
5        1e8zLHqP06aR0P5wmzn3vg  eK6FpEPv2urKpEjWwR45QQ   
6        atrZFItAcCVZxzGIaqAiqw  iTxX0v_NktTq9kpY26DeIA   
7        tJc8c_OiLyo6glPuPUAt0g  WI8aJ-Longj0nCQRUvAZQA   
8        0fx6T0DzabBDu9KJouaMVA  k6A2QvCacWCW_tB5TVZ5OQ   
9        RnzWdpOKHoayvaVdZcaHIQ  XY1l7g3axUCiiADD2zAEow   
10       ZdGrvp6Kq1hOt_3WPTOHhg  UNLdq6UCWtqDEQH5sm6zwg   
11       3y4QVNRb3

In [ ]:
from Word2VecUtility import Word2VecUtility

t0 = time.time()
rest_review_sents = []
# Cleaning and parsing the reviews...
for i in range( 0, len(restdata["text"])):
    # sent_reviews += Word2VecUtility.review_to_sentences(data["text"][i], tokenizer)
    rest_review_sents += Word2VecUtility.review_to_sentences(restdata.iloc[i]["text"])
    
t1 = time.time()
print(t1-t0)

In [30]:
len(rest_review_sents)

238272909

In [33]:
with open('rest_review_sents_first10m.pkl', 'wb') as out:
    pickle.dump(rest_review_sents[:10000000], out)

In [ ]:
with open('rest_review_sents_rest.pkl', 'wb') as out:
    pickle.dump(rest_review_sents[100000000:], out)

In [ ]:
from gensim.models import word2vec

t0 = time.time()
# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# continue to train the model (this will take some time)
model = Word2Vec.load("300features_40minwords_10context_1000000")

print("Training model...")
model.train(rest_review_sents, total_examples=len(rest_review_sents), epochs=model.epochs)

t1 = time.time()
print(t1-t0)


# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "300features_40minwords_10context_{}".format('all')
model.save(model_name)